In [90]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
#from bs4 import BeautifulSoup
from collections import defaultdict
#import requests
%matplotlib inline
from sklearn.svm import SVC
from nltk.stem.snowball import SnowballStemmer

## remove special symbol
def rm_sym(df):
    df['review'] = df['review'].str.replace("&#039;",'\'')
    df['review'].head()
    df['rating_cate'] = ''
    df.loc[df['rating'] >= 7,'rating_cate'] = 'high'
    df.loc[df['rating'] <= 4,'rating_cate'] = 'low'
    df.loc[(df['rating'] > 4) & (df['rating'] < 7),'rating_cate'] = 'medium'
    return df

def clean_text(df_tem3):
    df_tem3['review'] = df_tem3['review'].str.replace("\"","").str.lower()
    df_tem3['review'] = df_tem3['review'].str.replace( r"(\\r)|(\\n)|(\\t)|(\\f)|(\.)|(\;)|(\:)|(\!)|(\')|(\?)|(\,)|(\")|(\()|(\))|(\[)|(\])|(&#039;)|(\d\s)|(\d)|(\/)","")
    df_tem3['review'] = df_tem3['review'].str.replace("\"","").str.lower()
    df_tem3['review'] = df_tem3['review'].str.replace( r"(\$)|(\-)|(\\)|(\s{2,})"," ")
    df_tem3['review'].sample(1).iloc[0]

    stemmer = SnowballStemmer('english')
    df_tem3['review'] = df_tem3['review'].apply(lambda x: ' '.join([stemmer.stem(word) for word in x.split(" ")]))
    return df_tem3

In [121]:
df = pd.read_csv('drugsCom_raw/drugsComTrain_raw.tsv',sep='\t',index_col=0).sample(10000)
df = rm_sym(df)
df_tem3 = df

test = pd.read_csv("drugsCom_raw/drugsComTest_raw.tsv",sep='\t', index_col=0).sample(1000)
test = rm_sym(test)

df_tem3 = clean_text(df_tem3)
test = clean_text(test)

In [195]:
df_tem3.groupby('rating_cate').size()

rating_cate
high      6620
low       2469
medium     911
dtype: int64

In [122]:

print(df_tem3.shape)
print(test.shape)

(10000, 7)
(10000, 7)


In [123]:
import tensorflow as tf
import tensorflow 

#from tensorflow import tensorflow.keras

from keras.datasets import imdb
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.preprocessing.text import Tokenizer

# fix random seed for reproducibility

MAX_NB_WORDS = 3000
max_review_length = 500
EMBEDDING_DIM = 160


In [124]:
# Tokenize the data
tokenizer = Tokenizer(num_words = MAX_NB_WORDS, 
                      filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~',
                      lower=True, split=' ', char_level=False, 
                      oov_token=None, document_count=0)

tokenizer.fit_on_texts(df_tem3['review'])
train_sequences = tokenizer.texts_to_sequences(df_tem3['review'])
test_sequences = tokenizer.texts_to_sequences(test['review'])


In [125]:
# truncate and pad input sequences
X_train = sequence.pad_sequences(train_sequences, maxlen=max_review_length)
X_test = sequence.pad_sequences(test_sequences, maxlen = max_review_length)


In [126]:
y_train = pd.get_dummies(df_tem3['rating_cate'])
y_test = pd.get_dummies(test['rating_cate'])

word_index = tokenizer.word_index
y_train.head()

,high,low,medium
140021,1,0,0
131415,1,0,0
85719,1,0,0
1174,0,0,1
141852,0,0,1


In [127]:
# Print shapes of data. 

print(X_train.shape, '<-- shape of train_data ready for val/train split.')
print(X_test.shape, '<-- shape of final_test_data ready for fedding to network.')
print(len(tokenizer.word_index), '<-- Length of Word Index')

(10000, 500) <-- shape of train_data ready for val/train split.
(10000, 500) <-- shape of final_test_data ready for fedding to network.
13148 <-- Length of Word Index


In [128]:
# Split Training & Validation Data
from sklearn.model_selection import train_test_split


print('creating train and validation data by dividing train_data in 80:20 ratio')
######################################################

X_train_t, X_train_val, Y_train_t, y_train_val = train_test_split(X_train, y_train,test_size = 0.2)

######################################################
print('train data shape:', X_train_t.shape)
print('validation data shape:', X_train_val.shape)
print('Data is ready for training!!')

creating train and validation data by dividing train_data in 80:20 ratio
train data shape: (8000, 500)
validation data shape: (2000, 500)
Data is ready for training!!


In [129]:
nb_words  = min(MAX_NB_WORDS, len(word_index))
lstm_out = max_review_length

model = Sequential()
model.add(Embedding(nb_words,EMBEDDING_DIM,input_length=max_review_length))
model.add(LSTM(50))
#model.add(Attention(MAX_SEQUENCE_LENGTH))
model.add(Dense(3, activation = 'softmax'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [130]:
model.summary()


Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 500, 160)          16000     
_________________________________________________________________
unified_lstm_5 (UnifiedLSTM) (None, 50)                42200     
_________________________________________________________________
dense_5 (Dense)              (None, 3)                 153       
Total params: 58,353
Trainable params: 58,353
Non-trainable params: 0
_________________________________________________________________


In [131]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint


training_cycle = 1
batch = 32 
notebookname = "Drug_Data_"
variant = "LSTM_w_stopwords_"
version = "1.0_"
title = notebookname + variant + version


stamp = '{}trainging_cycle{}batchsize_{}'.format(title,training_cycle,batch)
print(stamp)
best_model_path = title + stamp + 'best.h5'

early_stopping = EarlyStopping(patience = 4)
model_checkpoint = ModelCheckpoint(best_model_path, save_best_only = True)


# Run LSTM Model
epoch = 40
LSTM_model = model.fit(X_train_t, Y_train_t, batch_size=batch, epochs=epoch,
                       validation_data=(X_train_val, y_train_val), shuffle = True, 
                       callbacks = [early_stopping, model_checkpoint], verbose = 0)
best_score = min(LSTM_model.history['val_loss'])


Drug_Data_LSTM_w_stopwords_1.0_trainging_cycle1batchsize_32


##### 

In [134]:
LSTM_model.history['val_accuracy']

[0.7896667,
 0.78500015,
 0.79650015,
 0.7953333,
 0.7994998,
 0.79933333,
 0.79466665,
 0.79899997,
 0.79566664,
 0.7911668,
 0.8008332,
 0.8033334,
 0.7893331,
 0.8063332,
 0.80533326,
 0.8041667]

In [136]:
## 1000
a= [0.7633334,
 0.765,
 0.7733334,
 0.7766667,
 0.79,
 0.78333336,
 0.77833337,
 0.77833337,
 0.78666663]



9

In [192]:
from sklearn.metrics import accuracy_score, confusion_matrix


y_test.head()

,high,low,medium
33287,0,1,0
53084,1,0,0
77802,1,0,0
120347,0,1,0
131030,0,1,0


In [179]:
truth = test['rating_cate'].apply(lambda x : 0 if x =='high' else (1 if x == 'low' else 2))
accuracy_score(truth,y_predict)

0.694

In [196]:
accr = model.evaluate(X_test,y_test, batch_size = 100)


10000/10000 [==============================] - 31s 3ms/sample - loss: 0.4698 - accuracy: 0.7990


In [184]:
prediction = model.predict(X_test)


In [214]:
model

In [203]:
accuracy_score(y_test.values.argmax(axis=1),prediction.argmax(axis=1))

0.694

In [209]:
confusion_matrix(y_test.values.argmax(axis=1),prediction.argmax(axis=1))

array([[5826,  681,   53],
       [1410, 1097,   37],
       [ 673,  206,   17]])

In [202]:

a = np.array([[10612,811,2074], [ 1440,1686,1703],[ 2370,1187,31883]])
(10612 + 1686+ 31883)/a.sum()

0.8217274857716773

In [201]:
a

array([[10612,   811,  2074],
       [ 1440,  1686,  1703],
       [ 2370,  1187, 31883]])